In [1]:
import os
import copy
import numpy as np
import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import running_as_notebook

In [2]:
def load_obj(filename):
    vertices = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('v '):
                parts = line.split()
                vertices.append([float(parts[1]), float(parts[2]), float(parts[3])])
    return np.array(vertices)

def get_dimensions(vertices):
    min_vals = np.min(vertices, axis=0)
    max_vals = np.max(vertices, axis=0)
    return max_vals - min_vals

In [3]:
def read_obj(
        obj_name,
        obj_position,
        obj_rotation,
        obj_mass = 0.5,
        obj_dir = os.path.join(os.getcwd(), "objects")
):
  obj_path = os.path.join(obj_dir, "{}.obj".format(obj_name))
  x, y, z = obj_position
  roll, pitch, yaw = obj_rotation
  obj_link = "file://{}".format(obj_path)
  # Load the OBJ file
  vertices = load_obj(obj_path)
  # Calculate dimensions
  dimensions = get_dimensions(vertices)
  width, height, depth = dimensions
  object_sdf = """
  <?xml version="1.0"?>
  <sdf version="1.7">
    <model name="{}">
      <link name="base_link_sugar">
        <inertial>
          <mass>{}</mass>
          <inertia>
            <ixx>0.001418</ixx>
            <ixy>0</ixy>
            <ixz>0</ixz>
            <iyy>0.000455</iyy>
            <iyz>0</iyz>
            <izz>0.001699</izz>
          </inertia>
        </inertial>
        <visual name="base_link_sugar">
          <pose>{} {} {} {} {} {}</pose>
          <geometry>
            <mesh>
              <uri>{}</uri>
            </mesh>
          </geometry>
        </visual>
      <collision name="box_collision">
          <geometry>
            <box>
              <size>{} {} {}</size>
            </box>
          </geometry>
          <drake:proximity_properties>
            <drake:compliant_hydroelastic/>
            <drake:hydroelastic_modulus>1.0e8</drake:hydroelastic_modulus>
          </drake:proximity_properties>
        </collision>
      </link>
    </model>
  </sdf>
  """.format(obj_name, obj_mass, x, y, z, roll, pitch, yaw, obj_link, width, height, depth)

  sdf_dir = os.path.join(os.getcwd(), "sdf")
  object_sdf_file = os.path.join(sdf_dir, "{}.sdf".format(obj_name))
  with open(object_sdf_file, "w") as f:
      f.write(object_sdf)
  obj_description = """- add_model:
    name: {}
    file: "file://{}"
""".format(obj_name, object_sdf_file)
  return [width, height, depth], obj_description

In [4]:
def read_sdf(obj_name, sdf_dir, obj_position, obj_rotation):
  x, y, z = obj_position
  roll, pitch, yaw = obj_rotation
  obj_path = os.path.join(sdf_dir, "{}.sdf".format(obj_name))
  sdf_link = "file://{}".format(obj_path)
  rotation = "deg: [{}, {}, {}]".format(roll, pitch, yaw)
  print(rotation)
  obj_description = """- add_model:
    name: {}
    file: "file://{}"
- add_weld:
    parent_body: world
    child: {}::base
    X_PC:
      translation: [{}, {}, {}].format(obj_name, sdf_link)
      rotation: !Rpy {{}}
""".format(obj_name, sdf_link, obj_name, x, y, z, rotation)
  return obj_description

In [6]:
table_name = "table"
table_bounds, table_description = read_obj(table_name, [0, 0, 0], [0, 0, 0])
box_name = "solid_box"
box_bounds, box_description = read_obj(box_name, [0, 0, table_bounds[2]], [0, 0, 0])

In [7]:
# num_objs = 3
# ycb_dir = os.path.join(os.getcwd(), "ycb")
# model_names = os.listdir(ycb_dir)
# selected_model_names = np.random.choice(model_names, num_objs, replace=False)
# for model_name in selected_model_names:
#     sdf_dir = os.path.join(ycb_dir, model_name)
#     files = os.listdir(sdf_dir)
#     for file in files:
#         if file.endswith(".sdf"):
#             obj_name = file.split(".")[0]
#             obj_description = read_sdf(obj_name, sdf_dir, [0, 0, table_bounds[2]], [0, 0, 0])
#             print(obj_description)
#             all_descriptions.append(obj_description)

In [8]:
yaml_file = """
directives:
{}
{}
""".format(table_description, box_description)
yaml_dir = os.path.join(os.getcwd(), "yaml")
if not os.path.exists(yaml_dir):
    os.makedirs(yaml_dir)
yaml_path = os.path.join(yaml_dir, "scene.dmd.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_file)

Should be added to another code

In [9]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [10]:
def setup_manipulation_station():
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            filename="file://{}".format(yaml_path),
            time_step=1e-3,
        )
    )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")

    MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )


    diagram = builder.Build()

    context = plant.CreateDefaultContext()


    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.01)



# Get initial pose of the gripper by using default context of manip station.
setup_manipulation_station()